In [0]:
pip install great_expectations

In [0]:
import pyspark.sql.functions as F
import datetime
import great_expectations as gx

In [0]:
df = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("/Volumes/workspace/default/grain_logistic_shipping")

In [0]:

context = gx.get_context()

data_source_name = "data-source-example"
data_source = context.data_sources.add_spark(name=data_source_name, persist=False)

data_asset_name = "example_data_asset"
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

batch_definition_name = "example_batch_definition"
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)

# Passamos o DataFrame que queremos validar para obter um "Batch" de dados
batch_parameters = {"dataframe": df}
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

print("Batch de dados criado. Iniciando validações...")

print("\n--- Validando Nulos ---")
for col_name in df.columns:
    expectation = gx.expectations.ExpectColumnValuesToNotBeNull(
        column=col_name
    )

    validation_results = batch.validate(expectation)
    
    if validation_results.success:
        print(f"✅ Nulidade da coluna '{col_name}' validada com sucesso!")
    else:
        
        print(f"❌ FALHA na validação de nulidade da coluna '{col_name}'")
        print(validation_results)
        
        # raise Exception(f"Falha na validação de nulos para a coluna {col_name}: {validation_results}")


print("\n--- Validando Duplicados na Chave Primária ---")
pk_column_name = "id_envio"
expectation = gx.expectations.ExpectColumnValuesToBeUnique(
    column=pk_column_name
)

validation_results = batch.validate(expectation)

if validation_results.success:
    print(f"✅ Unicidade da coluna '{pk_column_name}' validada com sucesso!")
else:
    print(f"❌ FALHA: Foram encontrados valores duplicados na coluna '{pk_column_name}'")
    print(validation_results)
    # raise Exception(f"Falha na validação de unicidade para a coluna {pk_column_name}: {validation_results}")

print("\nValidação concluída.")

In [0]:
df = df.withColumn("DtIngest", F.lit(data.strftime("%Y-%m-%d %H:%M:%S")))

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("grao.bronze.grain_shipping")